In [1]:
import pandas as pd
import argparse
import fasttext
from scipy.spatial import distance
import numpy as np
import io
import networkx as nx

lang1 = 'en'
lang2 = 'uk'
langs =[lang1,lang2]
pathVectors = 'vectors/'
pathAlignment = 'pathAlignment/'

In [ ]:
for l in langs:
    print(l)
    !wget -P vectors/ {'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.%s.zip' % l}

In [2]:
model = {}
for lang in langs:
    
    model[lang] = fasttext.load_model('wiki.%s.vec' % (lang))

ValueError: wiki.multi.en.vec has wrong file format!

In [ ]:
for l in langs:
    !unzip {'%s/wiki.%s.zip' % (pathVectors, l)}

In [63]:
from scipy.spatial import distance
a = model['uk'].get_sentence_vector("кіт")
b = model['en'].get_sentence_vector("cat")

distance.cosine(a, b)


NameError: name 'model' is not defined

In [37]:
def apply_transform(vec, transform):
        """
        Apply the given transformation to the vector space

        Right-multiplies given transform with embeddings E:
            E = E * transform

        Transform can either be a string with a filename to a
        text file containing a ndarray (compat. with np.loadtxt)
        or a numpy ndarray.
        """
        transmat = np.loadtxt(transform) if isinstance(transform, str) else transform
        return np.matmul(vec, transmat)

In [51]:
# Align second language
aAligned = apply_transform(a,'%sapply_in_%s_to_%s.txt' % (pathAlignment,'uk','en') )

In [52]:
distance.cosine(aAligned, b)


0.5366788424882428

In [53]:
a = model['uk'].get_word_vector("привіт")
b = model['en'].get_word_vector("hello")
distance.cosine(a, b)


0.9677601978182793

In [54]:
aAligned = apply_transform(a,'%sapply_in_%s_to_%s.txt' % (pathAlignment,'uk','en') )

distance.cosine(aAligned, b)

0.5312663719018984

In [55]:
a = model['en'].get_word_vector("hi")
b = model['en'].get_word_vector("hello")
distance.cosine(a, b)

0.402226984500885

In [2]:
def load_vec(emb_path, nmax=50000):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id



In [3]:
nmax = 1000000
src_embeddings_en, src_id2word_en, src_word2id_en = load_vec("wiki.multi.en.vec", nmax)

In [4]:
src_embeddings_uk, src_id2word_uk, src_word2id_uk = load_vec("wiki.multi.uk.vec", nmax)



In [ ]:
def get_nn(word, src_emb, src_id2word, tgt_emb, tgt_id2word, K=5):
    print("Nearest neighbors of \"%s\":" % word)
    word2id = {v: k for k, v in src_id2word.items()}
    word_emb = src_emb[word2id[word]]
    scores = (tgt_emb / np.linalg.norm(tgt_emb, 2, 1)[:, None]).dot(word_emb / np.linalg.norm(word_emb))
    k_best = scores.argsort()[-K:][::-1]
    for i, idx in enumerate(k_best):
        print('%.4f - %s' % (scores[idx], tgt_id2word[idx]))

In [ ]:
src_word = 'кіт'
get_nn(src_word, src_embeddings_uk, src_id2word_uk, src_embeddings_en, src_id2word_en, K=3)

In [8]:
import logging
from gensim.test.utils import common_corpus, common_dictionary, datapath
from gensim import corpora
from gensim.models import HdpModel, LdaMulticore, LdaModel
import pandas as pd
import argparse

lda_uk = LdaModel.load(datapath('lda_uk'))
lda_en = LdaModel.load(datapath('lda_en'))

In [ ]:
def show_topics(lda):
    for i in range(lda.num_topics):
        topic_terms = lda.get_topic_terms(topicid=i)
        print("Topic ", i)
        for term in topic_terms:
            print(lda.id2word[term[0]], " : ", term[1])



In [ ]:
show_topics(lda_en)

In [8]:
word2id_uk = {v: k for k, v in src_id2word_uk.items()}
vect1 = src_embeddings_uk[word2id_uk["кіт"]]

word2id_en = {v: k for k, v in src_id2word_en.items()}
vect2 = src_embeddings_en[word2id_en["cat"]]

distance.cosine(vect1, vect2)

0.49274123949257986

In [40]:
dict_uk = {}

In [12]:
from gensim.models.coherencemodel import CoherenceModel
# corpus = [lda_uk.id2word.doc2bow(text) for text in lda_uk.]

df = pd.read_json(f'preprocessed_dump_uk.json', orient='records')
texts = df['text'].tolist()
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]
cm = CoherenceModel(model=lda_uk, corpus=corpus, texts=texts)

In [13]:
cm.get_coherence()

0.6576220335975637

In [15]:
del texts, id2word, corpus


NameError: name 'texts' is not defined

In [16]:
del df

In [11]:
from gensim.models.coherencemodel import CoherenceModel
df = pd.read_json(f'preprocessed_dump_en.json', orient='records')
texts = df['text'].tolist()
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]
cm_en = CoherenceModel(model=lda_en, corpus=corpus, texts=texts)
cm_en.get_coherence()

0.5366094311625588

In [12]:
del df, texts, id2word, corpus, cm_en

In [15]:
import pyLDAvis.gensim

df = pd.read_json(f'preprocessed_dump_uk.json', orient='records')
texts = df['text'].tolist()
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_uk, corpus, dictionary=lda_uk.id2word)
vis

IndexError: index 1199135 is out of bounds for axis 1 with size 1199135

In [20]:
word2id_uk = {v: k for k, v in src_id2word_uk.items()}
i = 5
pairs = lda_uk.get_topic_terms(i)
for pair in pairs:
    word = lda_uk.id2word[pair[0]]
    print(word)
    try:
        emb = src_embeddings_uk[word2id_uk[word]]
    except:
        continue

        


муніципалітет
провінція
італія
схід
регіон
примітка
населення
список
захід
розташований


In [18]:
count = 0
word2id_en = {v: k for k, v in src_id2word_en.items()}
for i in range(lda_en.num_topics):
    pairs_en = lda_en.get_topic_terms(i)
    for pair in pairs_en:
        word = lda_en.id2word[pair[0]]
        try:
            emb = src_embeddings_en[word2id_en[word]]
        except:
            count += 1
            continue
print("exceptions : ", count)


exceptions :  0


In [19]:
i = 5
terms = lda_uk.get_topic_terms(i)
for term in terms:
    word = lda_uk.id2word[term[0]]
    probability = term[1]
    try:
        emb = src_embeddings_uk[word2id_uk[word]]
    except:
        continue
    


[(3293, 0.08925211),
 (3467, 0.07377604),
 (4775, 0.0320496),
 (3587, 0.028706985),
 (1096, 0.028052008),
 (211, 0.025943672),
 (983, 0.025624007),
 (4009, 0.025449486),
 (3134, 0.024663284),
 (2612, 0.024061227)]

In [9]:
def words_to_vectors(model, embeddings, src_id2word):
    topics = dict()
    word2id = {v: k for k, v in src_id2word.items()}
    print("num of topics : ", model.num_topics)
    for i in range(model.num_topics):
        terms = model.get_topic_terms(i)
        topic_terms = []
        for term in terms:
            word = model.id2word[term[0]]
            probability = term[1]
            try:
                emb = embeddings[word2id[word]]
                topic_terms.append([emb, probability])
            except:
                print("word not found")
        topics[i] = topic_terms
    return topics
            

In [10]:
en_topics = words_to_vectors(lda_en, src_embeddings_en, src_id2word_en)

num of topics :  40


In [11]:
len(en_topics)



40

In [12]:
uk_topics = words_to_vectors(lda_uk, src_embeddings_uk, src_id2word_uk)



num of topics :  40
word not found
word not found
word not found
word not found
word not found
word not found
word not found
word not found
word not found
word not found
word not found
word not found
word not found
word not found


In [13]:
def topics_distance(topic1, topic2):
    dist = 0
    for i in range(len(topic1)):
        dist = dist + distance.cosine(topic1[i][0], topic2[i][0])
    return dist

In [83]:
min_idx = 0
min_distance = 100
for i in en_topics:
   topic = en_topics[i]
   dist = topics_distance(uk_topics[4], topic)
   if dist < min_distance:
       min_idx = i
       min_distance = dist


In [82]:
topics_distance(uk_topics[0], en_topics[0])



8.419279278838296

In [84]:
min_idx

37

In [88]:
topic_terms = lda_uk.get_topic_terms(topicid=4)
for term in topic_terms:
    print(lda_uk.id2word[term[0]], " : ", term[1])

церква  :  0.010906376
святий  :  0.006578559
бог  :  0.004887131
монастир  :  0.004718315
свято  :  0.0045853527
православний  :  0.004357659
храм  :  0.004153548
собор  :  0.004079672
єпископ  :  0.0035496019
століття  :  0.0033643253


In [89]:
topic_terms = lda_en.get_topic_terms(topicid=37)
for term in topic_terms:
    print(lda_en.id2word[term[0]], " : ", term[1])


church  :  0.011229369
christian  :  0.005166045
jesus  :  0.004948357
century  :  0.0049105994
saint  :  0.004751941
christ  :  0.0041244915
time  :  0.003266497
roman  :  0.0032638793
say  :  0.0032421981
accord  :  0.0030428693


In [14]:
def find_closest_topic(topic, topics_list):
    min_idx = 0
    min_distance = 100
    for i in topics_list:
       t = topics_list[i]
       dist = topics_distance(topic, t)
       if dist < min_distance:
           min_idx = i
           min_distance = dist
    return min_idx, min_distance


In [15]:
def show_topic_terms(model, id):
    topic_terms = model.get_topic_terms(topicid=id)
    for term in topic_terms:
        print(model.id2word[term[0]], " : ", term[1])

In [16]:
show_topic_terms(lda_uk, 23)

український  :  0.01090142
армія  :  0.010775449
військо  :  0.0104261385
військовий  :  0.0097133
радянський  :  0.008174462
російський  :  0.00796036
війна  :  0.00766874
полк  :  0.006906269
україна  :  0.0061086034
генерал  :  0.004987835


In [22]:
uk_topic = uk_topics[23]
min_idx, min_dist = find_closest_topic(uk_topic, en_topics)
show_topic_terms(lda_en, min_idx)



force  :  0.011041672
army  :  0.007784993
military  :  0.007615827
state  :  0.005951281
battle  :  0.005930984
soviet  :  0.005286651
government  :  0.004951337
china  :  0.0044536702
british  :  0.004414635
attack  :  0.00428505


In [20]:
show_topic_terms(lda_uk, 25)

острів  :  0.013531725
річка  :  0.011641236
озеро  :  0.010140168
гора  :  0.009440495
частина  :  0.0094235875
південний  :  0.008294909
земля  :  0.007825516
північний  :  0.007621172
великий  :  0.0065237707
висота  :  0.006252185


In [21]:
uk_topic = uk_topics[25]
min_idx, min_dist = find_closest_topic(uk_topic, en_topics)
show_topic_terms(lda_en, min_idx)

island  :  0.010343966
water  :  0.00896965
islands  :  0.007887532
north  :  0.0069999634
south  :  0.0062251817
land  :  0.005586973
river  :  0.0053218524
species  :  0.0051699486
area  :  0.004390231
include  :  0.0043121492
